In [41]:
import pandas as pd
import numpy as np

# 假设有1000个用户和20个商品
n_users = 10000
n_items = 10

# 创建用户和商品的ID
users = np.random.randint(0, n_users, size=10000)
items = np.random.randint(0, n_items, size=10000)

# 创建一个简单的用户行为数据集，这里仅用用户ID和商品ID
data = pd.DataFrame({'user_id': users, 'item_id': items})

# 分割训练集和测试集
train_data = data.sample(frac=0.8, random_state=123)
test_data = data.drop(train_data.index)


In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, Concatenate
from tensorflow.keras.models import Model

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, hidden_units):
        super(AttentionLayer, self).__init__()
        self.hidden_units = hidden_units
        self.W_query = tf.keras.layers.Dense(hidden_units, activation=tf.nn.relu, name="attention_W_query")
        self.W_key = tf.keras.layers.Dense(hidden_units, activation=tf.nn.relu, name="attention_W_key")
        self.V = tf.keras.layers.Dense(1, activation=None, name="attention_V")

    def call(self, queries, keys):
        queries = self.W_query(queries)
        keys = self.W_key(keys)
        attention_scores = self.V(tf.nn.tanh(queries + keys))
        attention_weights = tf.nn.softmax(attention_scores, axis=1)
        output = tf.reduce_sum(attention_weights * keys, axis=1)
        return output

def get_din_model(n_users, n_items, embedding_size):
    # 输入层
    user_id_input = Input(shape=(1,), name='user_id')
    item_id_input = Input(shape=(1,), name='item_id')
    hist_item_input = Input(shape=(10,), name='hist_item_id')  # 历史行为长度设为10

    # 嵌入层
    embedding_layer = Embedding(n_items, embedding_size, input_length=10, name='item_embedding')
    user_embedding = Embedding(n_users, embedding_size, input_length=1, name='user_embedding')

    # 用户和商品的嵌入
    user_embed = user_embedding(user_id_input)
    item_embed = embedding_layer(item_id_input)
    hist_item_embed = embedding_layer(hist_item_input)

    # DIN的注意力机制
    attention_layer = AttentionLayer(hidden_units=32)
    hist_item_attention = attention_layer(hist_item_embed, user_embed)

    # Flatten嵌入向量
    user_embed = Flatten()(user_embed)
    item_embed = Flatten()(item_embed)
    hist_item_attention = Flatten()(hist_item_attention)

    # 将嵌入向量连接在一起
    attention_input = Concatenate()([hist_item_attention, user_embed, item_embed])

    # 输出层
    output = Dense(1, activation='sigmoid')(attention_input)

    # 构建模型
    model = Model(inputs=[user_id_input, item_id_input, hist_item_input], outputs=output)
    return model

# 创建模型实例
embedding_size = 8
model = get_din_model(n_users, n_items, embedding_size)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [80]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 hist_item_id (InputLayer)      [(None, 10)]         0           []                               
                                                                                                  
 user_id (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 item_embedding (Embedding)     multiple             80          ['item_id[0][0]',                
                                                                  'hist_item_id[0][0]']           
                                                                                                  
 user_embedding (Embedding)     (None, 1, 8)         80000       ['user_id[0][0]']            

In [42]:
train_hist_items

array([[9, 7, 1, ..., 0, 9, 1],
       [6, 9, 8, ..., 7, 7, 9],
       [3, 8, 1, ..., 8, 4, 4],
       ...,
       [8, 6, 1, ..., 3, 8, 6],
       [8, 0, 1, ..., 4, 6, 7],
       [5, 5, 6, ..., 9, 5, 2]])

In [74]:
# 准备训练数据
train_user_ids = np.array(train_data['user_id'])
train_item_ids = np.array(train_data['item_id'])
train_hist_items = np.random.randint(0, n_items, size=(len(train_data), 10))

# 随机生成标签
train_labels = np.random.randint(0, 2, size=(len(train_data), 1))

# 训练模型
model.fit([train_user_ids, train_item_ids, train_hist_items], train_labels, epochs=100, batch_size=32)


Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7574 - accuracy: 0.5089
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6135 - accuracy: 0.7410
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5189 - accuracy: 0.7990
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4153 - accuracy: 0.8159
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8184
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3023 - accuracy: 0.8171
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2828 - accuracy: 0.8201
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2710 - accuracy: 0.8173
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2636 - accuracy: 0.8209
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2582 - accura

250/250 [==============================] - 0s 2ms/step - loss: 0.2252 - accuracy: 0.8374
Epoch 82/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2254 - accuracy: 0.8438
Epoch 83/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2259 - accuracy: 0.8380
Epoch 84/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2249 - accuracy: 0.8422
Epoch 85/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2260 - accuracy: 0.8413
Epoch 86/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2252 - accuracy: 0.8422
Epoch 87/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2256 - accuracy: 0.8369
Epoch 88/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2251 - accuracy: 0.8451
Epoch 89/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2250 - accuracy: 0.8390
Epoch 90/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2247 - accuracy: 

In [75]:
# 准备测试数据
test_user_ids = np.array(train_data['user_id'])
test_item_ids = np.array(train_data['item_id'])
test_hist_items = np.random.randint(0, n_items, size=(len(train_data), 10))

# 随机生成测试标签
test_labels = np.random.randint(0, 2, size=(len(train_data), 1))

# 评估模型
evaluation = model.evaluate([test_user_ids, test_item_ids, test_hist_items], test_labels)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


250/250 [==============================] - 0s 2ms/step - loss: 2.8721 - accuracy: 0.5013
Test Loss: 2.8721251487731934, Test Accuracy: 0.5012500286102295


In [76]:
# 准备测试数据
test_user_ids = np.array(test_data['user_id'])
test_item_ids = np.array(test_data['item_id'])
test_hist_items = np.random.randint(0, n_items, size=(len(test_data), 10))

# 随机生成测试标签
test_labels = np.random.randint(0, 2, size=(len(test_data), 1))

# 评估模型
evaluation = model.evaluate([test_user_ids, test_item_ids, test_hist_items], test_labels)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


63/63 [==============================] - 0s 1ms/step - loss: 2.0905 - accuracy: 0.5060
Test Loss: 2.090545177459717, Test Accuracy: 0.5059999823570251


In [77]:

# 预测点击率
# 这里假设我们有一个包含所有项目ID的数组
item_ids = np.array(range(n_items)).reshape(-1, 1)
user_ids = np.array([test_user_ids[3]] * n_items)  # 某个特定用户的ID
hist_items = np.random.randint(0, 10, size=(10, 10))

# 预测每个项目的点击概率
click_probabilities = model.predict([user_ids, item_ids, hist_items])

# 输出结果
for item_id, prob in zip(item_ids, click_probabilities):
    print(f"Item ID {item_id}: Click Probability {prob}")


1/1 [==============================] - 0s 52ms/step
Item ID [0]: Click Probability [0.45413792]
Item ID [1]: Click Probability [0.4943115]
Item ID [2]: Click Probability [0.46201047]
Item ID [3]: Click Probability [0.47057247]
Item ID [4]: Click Probability [0.44858602]
Item ID [5]: Click Probability [0.4325587]
Item ID [6]: Click Probability [0.5215335]
Item ID [7]: Click Probability [0.4312032]
Item ID [8]: Click Probability [0.44681546]
Item ID [9]: Click Probability [0.43566415]


In [1]:
import tensorflow as tf
from tensorflow.keras import layers